# Uncertainty Engine SDK - saving and loading workflows

This notebook goes through some of the basic functionality of the SDK. It demonstrates how to save and load your workflows.


## The `Client`

Start by importing the `Client`.


In [ ]:
from uncertainty_engine import Client, Environment

Next initialise the `Client` with the Uncertainty Engine environment to target.

In [ ]:
client = Client(
    env=Environment(
        cognito_user_pool_client_id="<COGNITO USER POOL APPLICATION CLIENT ID>",
        core_api="<UNCERTAINTY ENGINE CORE API URL>",
        region="<REGION>",
        resource_api="<UNCERTAINTY ENGINE RESOURCE SERVICE API URL>",
    ),
)

Before accessing your workflows you must authenticate with your account ID and select the project you wish to save and load from.

In [ ]:
# Authenticate using your account ID
client.authenticate("YOUR_ACCOUNT_ID")

# Select your project you wish to save and load from using your selected project ID
PROJECT_ID = "YOUR_PROJECT_ID"

## Saving a new workflow

Firstly, we will create a basic add workflow to save. 

In [ ]:
from uncertainty_engine.graph import Graph
from uncertainty_engine.nodes.basic import Add

graph = Graph()

add = Add(rhs=1, lhs=42, label="Add")

# Add the node to the graph
graph.add_node(add)

# Define our add output handle
ans = add.make_handle("ans")

Make sure to wrap your basic workflow in the `Workflow` node before saving. The save method requires the workflow you wish to save to be an instance of the `Workflow` class.

In [ ]:
from uncertainty_engine.nodes.workflow import Workflow
from pprint import pprint

basic_workflow = Workflow(
    graph=graph.nodes,
    input=graph.external_input,
    external_input_id=graph.external_input_id,
    requested_output={"Add": ans.model_dump()},
)

Now we can save our workflow. As it is a new workflow you will need to make sure you pass it a `workflow_name`. This must be unique to your other saved workflows.

You can tell your workflow has been saved successfully if your workflow ID is returned.

In [6]:
basic_workflow_id = client.workflows.save(
    project_id=PROJECT_ID,
    workflow=basic_workflow,
    workflow_name="MyBasicAddWorkflow",
)

print(basic_workflow_id)

684ffd7bd465c24bf9e9e384


## Listing your existing workflows

We can now list our workflows using the `list_workflows` method to see our saved workflow details.

In [7]:
pprint(client.workflows.list_workflows(PROJECT_ID))

[{'created_at': '11:18:19 2025-06-16',
  'id': '684ffd7bd465c24bf9e9e384',
  'name': 'MyBasicAddWorkflow',
  'versions': ['684ffd7bd465c24bf9e9e385']}]


You will see that along side the ID this method returns the name, list of versions and time the workflow was created. 

This is a particularly useful method when you start having many workflows and would like to view which one you wish to load.

## Loading your saved workflow

When loading a resource you must provide the `project_id` and the `workflow_id`. Optionally, if you wish to load a specific previous version of your workflow, you can provide the `version_id` as well. Otherwise, the latest version of your workflow will be loaded.

In [ ]:
loaded_workflow = client.workflows.load(
    project_id=PROJECT_ID, workflow_id=basic_workflow_id
)

Now we have loaded our basic add workflow we can run it as shown below.

In [9]:
response = client.run_node(loaded_workflow)
pprint(response)

{'inputs': {'external_input_id': '_',
            'graph': {'nodes': {'Add': {'inputs': {'lhs': {'node_handle': 'Add_lhs',
                                                           'node_name': '_'},
                                                   'rhs': {'node_handle': 'Add_rhs',
                                                           'node_name': '_'}},
                                        'type': 'Add'}}},
            'inputs': {'Add_lhs': 42, 'Add_rhs': 1},
            'requested_output': {'Add': {'node_handle': 'ans',
                                         'node_name': 'Add'}}},
 'message': 'Job completed at 2025-06-16 11:18:29.386789',
 'outputs': {'outputs': {'Add': 43.0}},
 'progress': {'Add': {'inputs': {'lhs': 42, 'rhs': 1},
                      'message': 'Job completed at 2025-06-16 11:18:29.022477',
                      'outputs': {'ans': 43.0},
                      'progress': None,
                      'status': 'completed'}},
 'status': 'completed'}


## Creating a new version of an existing workflow

When we wish to save a new version of an already existing workflow we can simply use the `save` method but make sure to provide the `workflow_id` under which you wish the new version to be saved.

Note that when saving an already existing workflow, the `workflow_name` is not required.

In [ ]:
client.workflows.save(
    project_id=PROJECT_ID,
    workflow=basic_workflow,
    workflow_id=basic_workflow_id,
)

# List all workflows in the project
pprint(client.workflows.list_workflows(PROJECT_ID))

[{'created_at': '11:18:19 2025-06-16',
  'id': '684ffd7bd465c24bf9e9e384',
  'name': 'MyBasicAddWorkflow',
  'versions': ['684ffd7bd465c24bf9e9e385', '684ffd87d465c24bf9e9e386']}]


As you can see above `MyBasicAddWorkflow` now has another version in its list of versions.